# 신경망 학습의 수학적 기초 - 실습 노트북 (기본 단계)

이 노트북은 신경망의 기본 개념을 수치 예제와 실행 가능한 코드로 검증합니다.

## 목차
1. 퍼셉트론 구현
2. 활성화 함수 시각화
3. 순방향 전파 구현
4. 손실 함수 계산

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 한글 폰트 설정 (선택적)
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

print("NumPy version:", np.__version__)

## 1. 퍼셉트론 (Perceptron)

### 수식
$$y = f(w^T x + b)$$

**기호 설명:**
- $x$: 입력 벡터
- $w$: 가중치 벡터
- $b$: 편향
- $f(\cdot)$: 활성화 함수

In [ ]:
# 퍼셉트론 구현
def perceptron(x, w, b):
    """
    단순 퍼셉트론
    
    Parameters:
    -----------
    x : array-like, 입력 벡터
    w : array-like, 가중치 벡터
    b : float, 편향
    
    Returns:
    --------
    y : float, 출력 (0 또는 1)
    """
    z = np.dot(w, x) + b  # 가중합
    y = 1 if z > 0 else 0  # 계단 함수
    return y, z

# 수치 예제 검증
x = np.array([2.0, 3.0])
w = np.array([0.5, 0.3])
b = 0.1

y, z = perceptron(x, w, b)

print("=" * 50)
print("퍼셉트론 수치 예제")
print("=" * 50)
print(f"입력 x: {x}")
print(f"가중치 w: {w}")
print(f"편향 b: {b}")
print()
print("계산 과정:")
print(f"  w^T x = {w[0]} × {x[0]} + {w[1]} × {x[1]}")
print(f"        = {w[0] * x[0]} + {w[1] * x[1]}")
print(f"        = {np.dot(w, x)}")
print(f"  z = w^T x + b = {np.dot(w, x)} + {b} = {z}")
print(f"  y = f(z) = {y} (z > 0 이므로)")
print()
print(f"최종 출력: {y}")
print("=" * 50)

## 2. 활성화 함수 (Activation Functions)

### 2.1 Sigmoid 함수

**수식:** $$\sigma(z) = \frac{1}{1 + e^{-z}}$$

**미분:** $$\frac{d\sigma(z)}{dz} = \sigma(z) \times (1 - \sigma(z))$$

In [ ]:
def sigmoid(z):
    """Sigmoid 활성화 함수"""
    return 1 / (1 + np.exp(-z))

def sigmoid_derivative(z):
    """Sigmoid 함수의 미분"""
    s = sigmoid(z)
    return s * (1 - s)

# 수치 예제
test_values = [0, 2, -2]

print("=" * 50)
print("Sigmoid 함수 수치 예제")
print("=" * 50)
for z in test_values:
    s = sigmoid(z)
    print(f"σ({z:2}) = {s:.4f}")
print("=" * 50)

In [ ]:
# 활성화 함수 시각화
z = np.linspace(-6, 6, 200)

# Sigmoid
y_sigmoid = sigmoid(z)
dy_sigmoid = sigmoid_derivative(z)

# ReLU
y_relu = np.maximum(0, z)
dy_relu = np.where(z > 0, 1, 0)

# Tanh
y_tanh = np.tanh(z)
dy_tanh = 1 - np.tanh(z)**2

# 플롯
fig, axes = plt.subplots(2, 3, figsize=(15, 8))
fig.suptitle('Activation Functions and Their Derivatives', fontsize=16)

# Sigmoid
axes[0, 0].plot(z, y_sigmoid, 'b-', linewidth=2)
axes[0, 0].set_title('Sigmoid')
axes[0, 0].set_xlabel('z')
axes[0, 0].set_ylabel('σ(z)')
axes[0, 0].grid(True, alpha=0.3)

axes[1, 0].plot(z, dy_sigmoid, 'r-', linewidth=2)
axes[1, 0].set_title('Sigmoid Derivative')
axes[1, 0].set_xlabel('z')
axes[1, 0].set_ylabel("σ'(z)")
axes[1, 0].grid(True, alpha=0.3)

# ReLU
axes[0, 1].plot(z, y_relu, 'g-', linewidth=2)
axes[0, 1].set_title('ReLU')
axes[0, 1].set_xlabel('z')
axes[0, 1].set_ylabel('ReLU(z)')
axes[0, 1].grid(True, alpha=0.3)

axes[1, 1].plot(z, dy_relu, 'r-', linewidth=2)
axes[1, 1].set_title('ReLU Derivative')
axes[1, 1].set_xlabel('z')
axes[1, 1].set_ylabel("ReLU'(z)")
axes[1, 1].grid(True, alpha=0.3)

# Tanh
axes[0, 2].plot(z, y_tanh, 'm-', linewidth=2)
axes[0, 2].set_title('Tanh')
axes[0, 2].set_xlabel('z')
axes[0, 2].set_ylabel('tanh(z)')
axes[0, 2].grid(True, alpha=0.3)

axes[1, 2].plot(z, dy_tanh, 'r-', linewidth=2)
axes[1, 2].set_title('Tanh Derivative')
axes[1, 2].set_xlabel('z')
axes[1, 2].set_ylabel("tanh'(z)")
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. 순방향 전파 (Forward Propagation)

### 수식
층 $l$에서의 연산:
$$z^{[l]} = W^{[l]} a^{[l-1]} + b^{[l]}$$
$$a^{[l]} = f^{[l]}(z^{[l]})$$

**기호 설명:**
- $W^{[l]}$: $l$번째 층의 가중치 행렬
- $a^{[l-1]}$: 이전 층의 활성화 값
- $b^{[l]}$: $l$번째 층의 편향 벡터
- $z^{[l]}$: 선형 결합
- $f^{[l]}(\cdot)$: 활성화 함수

In [ ]:
def relu(z):
    """ReLU 활성화 함수"""
    return np.maximum(0, z)

def forward_propagation(a0, W1, b1, W2, b2):
    """
    2층 신경망의 순방향 전파
    
    Parameters:
    -----------
    a0 : 입력 벡터
    W1, b1 : 1층 파라미터
    W2, b2 : 2층 파라미터
    
    Returns:
    --------
    cache : 중간 값들을 담은 딕셔너리
    """
    # 1층 (은닉층, ReLU)
    z1 = np.dot(W1, a0) + b1
    a1 = relu(z1)
    
    # 2층 (출력층, Sigmoid)
    z2 = np.dot(W2, a1) + b2
    a2 = sigmoid(z2)
    
    cache = {
        'a0': a0, 'z1': z1, 'a1': a1,
        'z2': z2, 'a2': a2
    }
    
    return cache

# 수치 예제 (문서의 예제와 동일)
a0 = np.array([1.0, 2.0])
W1 = np.array([[0.5, 0.3],
               [0.2, 0.4]])
b1 = np.array([0.1, 0.2])
W2 = np.array([[0.6, 0.7]])
b2 = np.array([0.15])

cache = forward_propagation(a0, W1, b1, W2, b2)

print("=" * 60)
print("순방향 전파 수치 예제")
print("=" * 60)
print(f"입력 a^[0]: {cache['a0']}")
print()
print("1층 (은닉층):")
print(f"  W^[1] = \n{W1}")
print(f"  b^[1] = {b1}")
print(f"  z^[1] = W^[1] a^[0] + b^[1] = {cache['z1']}")
print(f"  수동 계산: z1[0] = {W1[0,0]}×{a0[0]} + {W1[0,1]}×{a0[1]} + {b1[0]} = {W1[0,0]*a0[0] + W1[0,1]*a0[1] + b1[0]}")
print(f"  수동 계산: z1[1] = {W1[1,0]}×{a0[0]} + {W1[1,1]}×{a0[1]} + {b1[1]} = {W1[1,0]*a0[0] + W1[1,1]*a0[1] + b1[1]}")
print(f"  a^[1] = ReLU(z^[1]) = {cache['a1']}")
print()
print("2층 (출력층):")
print(f"  W^[2] = {W2}")
print(f"  b^[2] = {b2}")
print(f"  z^[2] = W^[2] a^[1] + b^[2] = {cache['z2']}")
print(f"  수동 계산: z2 = {W2[0,0]}×{cache['a1'][0]} + {W2[0,1]}×{cache['a1'][1]} + {b2[0]} = {W2[0,0]*cache['a1'][0] + W2[0,1]*cache['a1'][1] + b2[0]}")
print(f"  a^[2] = σ(z^[2]) = {cache['a2']}")
print()
print(f"최종 출력 (예측값 ŷ): {cache['a2'][0]:.4f}")
print("=" * 60)

## 4. 손실 함수 (Loss Functions)

### 4.1 평균 제곱 오차 (MSE)

**수식:** $$L(y, \hat{y}) = \frac{1}{2}(y - \hat{y})^2$$

### 4.2 교차 엔트로피 (Cross-Entropy)

**이진 분류:** $$L(y, \hat{y}) = -[y \log(\hat{y}) + (1-y) \log(1-\hat{y})]$$

In [ ]:
def mse_loss(y_true, y_pred):
    """평균 제곱 오차"""
    return 0.5 * (y_true - y_pred) ** 2

def binary_cross_entropy(y_true, y_pred):
    """이진 교차 엔트로피"""
    epsilon = 1e-10  # log(0) 방지
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    return -(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

# 수치 예제
print("=" * 60)
print("손실 함수 수치 예제")
print("=" * 60)

# MSE 예제
y_true = 1.0
y_pred = 0.8
loss = mse_loss(y_true, y_pred)
print("\nMSE 손실:")
print(f"  실제값: y = {y_true}")
print(f"  예측값: ŷ = {y_pred}")
print(f"  L = (1/2)(y - ŷ)² = (1/2)({y_true} - {y_pred})² = {loss}")

# 교차 엔트로피 예제
print("\n교차 엔트로피 손실:")
test_cases = [
    (1, 0.9),
    (0, 0.2),
    (1, 0.5)
]

for y, y_hat in test_cases:
    loss = binary_cross_entropy(y, y_hat)
    print(f"  y={y}, ŷ={y_hat}: L = {loss:.4f}")

print("=" * 60)

## 5. 완전한 예제: 앞의 순방향 전파 결과로 손실 계산

In [ ]:
# 앞의 순방향 전파 결과 사용
y_pred = cache['a2'][0]
y_true = 1.0

loss = binary_cross_entropy(y_true, y_pred)

print("=" * 60)
print("완전한 예제: 순방향 전파 + 손실 계산")
print("=" * 60)
print(f"예측값 (순방향 전파 결과): ŷ = {y_pred:.4f}")
print(f"실제값: y = {y_true}")
print(f"손실 (교차 엔트로피): L = {loss:.4f}")
print()
print("수동 계산:")
print(f"  L = -[y log(ŷ) + (1-y) log(1-ŷ)]")
print(f"    = -[{y_true} × log({y_pred:.4f}) + 0 × log({1-y_pred:.4f})]")
print(f"    = -log({y_pred:.4f})")
print(f"    = {-np.log(y_pred):.4f}")
print("=" * 60)

## 6. 손실 함수 시각화

In [ ]:
# 예측값 범위
y_pred_range = np.linspace(0.01, 0.99, 100)

# y=1일 때와 y=0일 때의 손실
loss_y1 = binary_cross_entropy(1, y_pred_range)
loss_y0 = binary_cross_entropy(0, y_pred_range)

# MSE 비교
mse_y1 = mse_loss(1, y_pred_range)
mse_y0 = mse_loss(0, y_pred_range)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 교차 엔트로피
axes[0].plot(y_pred_range, loss_y1, 'b-', linewidth=2, label='y=1')
axes[0].plot(y_pred_range, loss_y0, 'r-', linewidth=2, label='y=0')
axes[0].set_xlabel('Predicted Probability (ŷ)', fontsize=12)
axes[0].set_ylabel('Cross-Entropy Loss', fontsize=12)
axes[0].set_title('Binary Cross-Entropy Loss', fontsize=14)
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)

# MSE
axes[1].plot(y_pred_range, mse_y1, 'b-', linewidth=2, label='y=1')
axes[1].plot(y_pred_range, mse_y0, 'r-', linewidth=2, label='y=0')
axes[1].set_xlabel('Predicted Value (ŷ)', fontsize=12)
axes[1].set_ylabel('MSE Loss', fontsize=12)
axes[1].set_title('Mean Squared Error Loss', fontsize=14)
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 요약

이 노트북에서 다룬 내용:

1. **퍼셉트론**: 신경망의 기본 단위 구현 및 수치 검증
2. **활성화 함수**: Sigmoid, ReLU, Tanh의 구현 및 시각화
3. **순방향 전파**: 2층 신경망을 통한 완전한 순방향 전파 과정
4. **손실 함수**: MSE와 교차 엔트로피의 계산 및 비교

### 다음 단계

다음 노트북에서는 **역전파(Backpropagation)** 알고리즘을 구현하여 신경망을 실제로 학습시키는 방법을 배웁니다.